In [3]:
import json
import os
from PyPDF2 import PdfReader
import os
import requests
import json
from bs4 import BeautifulSoup
import pymysql

#PDF 다운로드
def download(url, file_path):
    with open(file_path, "wb") as file:   
        response = requests.get(url)               
        file.write(response.content)

In [27]:
import os
import requests
from bs4 import BeautifulSoup
import pymysql
from PyPDF2 import PdfReader
import json
import sys

# 재귀 깊이 설정
sys.setrecursionlimit(1500)

# DB 접근
db = pymysql.connect(host='paper-project-1.cne82okq8qfy.us-east-2.rds.amazonaws.com', 
                     port=3306, user="choi", passwd="UbcFp1C!7^shuxKYv4Q$", 
                     db="PAPER_PROJECT", charset="utf8")
cursor = db.cursor()

# 다운로드 함수 정의
def download(link, file_path):
    response = requests.get(link)
    with open(file_path, 'wb') as f:
        f.write(response.content)

# 특수 문자 제거 함수 정의
def sanitize_filename(filename):
    return "".join([c if c.isalnum() or c in (' ', '.', '_') else '_' for c in filename])

# 페이지에서 각 PDF URL 전달, title, media, date 변수 설정 
for i in range(101, 133): #1~132page -> 133까지
    url = f'https://finance.naver.com/research/debenture_list.naver?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2008-04-01&writeToDate=2021-05-26&x=33&y=12&page={i}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    rows = soup.select('#contentarea_left > div.box_type_m > table.type_1 > tr:has(> td:nth-child(4))')

    for li in rows:
        tds = [td.text.strip() for td in li.select('td')]
        if li.select('td:nth-child(3) > a'):        
            link = li.select('a')[1]['href']
        else:
            continue
        title, media, _, date, view = tds
        title = sanitize_filename(title)  # 파일 이름에서 특수 문자 제거
        print(title, media, link, date, view)

        directory = "/Users/choihj/web/date_report"  # PDF 파일 저장 경로
        file_path = os.path.join(directory, f'{title}.pdf')

        # PDF 다운로드
        try:
            download(link, file_path)
        except Exception as e:
            print(f"Failed to download {link}: {e}")
            continue

        # PDF 파일 읽기 및 텍스트 추출
        try:
            reader = PdfReader(file_path, strict=False)
            pages = reader.pages
            text = []
            for page in pages:
                try:
                    sub = page.extract_text()
                    if sub:
                        text.append(sub)
                except Exception as e:
                    print(f"Failed to extract text from {title}: {e}")
                    continue
        except RecursionError as e:
            print(f"RecursionError in file {title}: {e}")
            continue
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            continue

        # JSON 변환
        contents = {"keyword": text}
        contents = json.dumps(contents)

        # SQL에 데이터 삽입
        sql = "INSERT INTO text_analysis (document_type, date, title,media, sentences, sentiment_score_market, reg_date) VALUES (%s, %s,%s, %s, %s, %s, %s);"
        cursor.execute(sql, ('analyst_report', date, title,media, contents, 0,'2024-09-02'))
        db.commit()

db.close()


당일 채권시장은 미국 채권시장 약세 및 3월 수입물가.. IBK투자증권 http://imgstock.naver.com/upload/research/debenture/1302833634628.pdf 11.04.15 133
당일 채권시장은 미국 채권시장 강세 및 가격부담에 .. IBK투자증권 http://imgstock.naver.com/upload/research/debenture/1302747457518.pdf 11.04.14 129
당일 채권시장은 통화정책 불확실성 해소 및 미국 채.. IBK투자증권 http://imgstock.naver.com/upload/research/debenture/1302658776286.pdf 11.04.13 132
전격적인 긴축 가능성_ 유동성 사정이 달라질 조짐 키움증권 http://imgstock.naver.com/upload/research/debenture/1323132064627.pdf 11.04.12 140
당일 채권시장은 금통위에 따른 통화정책 경계감으로 .. IBK투자증권 http://imgstock.naver.com/upload/research/debenture/1302574294881.pdf 11.04.12 132
당일 채권시장은 4월 금통위를 앞둔 통화정책 경계감.. IBK투자증권 http://imgstock.naver.com/upload/research/debenture/1302500769281.pdf 11.04.11 134
당일 채권시장은 우호적인 수급 및 4월 금통위를 앞둔.. IBK투자증권 http://imgstock.naver.com/upload/research/debenture/1302245408518.pdf 11.04.08 161
채권시장은 미국 채권시장 약세 및 중국 기준금리 인.. IBK투자증권 http://imgstock.naver.com/upload/research/debenture/1302054065050.pdf 11.04.06 165
4일 국내 채권시장은 금통위 전까지 큰 변동없는 장

In [7]:
       
#DB 접근

# db=pymysql.connect(host='localhost', port=3306, user="study", passwd="study", db="study", charset="utf8")
db=pymysql.connect(host='paper-project-1.cne82okq8qfy.us-east-2.rds.amazonaws.com', port=3306, user="choi", passwd="UbcFp1C!7^shuxKYv4Q$", db="PAPER_PROJECT", charset="utf8")
cursor = db.cursor()

#페이지에서 각 pdf url 전달, title, media, date 변수 설정 
for i in range(1, 105):
    url=f'https://finance.naver.com/research/debenture_list.naver?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2008-04-01&writeToDate=2017-12-31&x=33&y=12&page={i}'
    res=requests.get(url)
    soup=BeautifulSoup(res.text,'html.parser')
    rows=soup.select('#contentarea_left > div.box_type_m > table.type_1> tr:has(> td:nth-child(4))')
    for li in rows:
        tds=[td.text for td in li.select('td')]
        link=li.select('a')[1]['href']
        title, media, _, date, view = tds
        print(title, media, link, date, view)
        directory = "/Users/choihj/web/date_report" #pdf파일 해당 경로에 저장
        file_path = os.path.join(directory,f'{title}.pdf') #파일 조인
        download(link,file_path) #download 함수 호출
        
        
        reader=PdfReader(f'/Users/choihj/web/date_report/{title}.pdf', strict=False) #pdf 읽어와서 Pdfreader 라이브러리에 넣기
        pages=reader.pages #페이지화
        
        text=[]
        for page in pages: #각 페이지 별로 텍스트 변환
            try:
                sub=page.extract_text()
                text.append(sub)
            except: #몇몇 파일이 깨져서 텍스트로 변환하지 못함. 그래서 try except 추가
                continue
        
        #json 변환
        contents = {"keyword": text} 
        contents = json.dumps(contents)
        
        #SQL에 데이터 넣기
        sql = "INSERT INTO text_analysis (document_type,date,title,sentences,sentiment_score_market,reg_date) VALUES (%s, %s,%s,%s,%s,%s);"
        cursor.execute(sql, ('analyst_report','2024-09-02',title,contents,0,date))
        db.commit()

트럼프노믹스를 극복한 2017년 시장  하나증권 http://imgstock.naver.com/upload/research/debenture/1514339254450.pdf 17.12.27 409
11월 채권시장 전망 현대차증권 http://imgstock.naver.com/upload/research/debenture/1514252144509.pdf 17.12.26 369
2금융권 안심전환대출 수급영향 점검  하나증권 http://imgstock.naver.com/upload/research/debenture/1513734929105.pdf 17.12.20 329
확신 없는 인상의견도 있었음이 확인 현대차증권 http://imgstock.naver.com/upload/research/debenture/1513733575212.pdf 17.12.20 292
속도 조절 중 현대차증권 http://imgstock.naver.com/upload/research/debenture/1513560615141.pdf 17.12.18 370
성장과 물가의 Dilemma 교보증권 http://imgstock.naver.com/upload/research/debenture/1513303183819.pdf 17.12.15 350
12월 FOMC 함의와 2018년 연준구성 변화  하나증권 http://imgstock.naver.com/upload/research/debenture/1513302272756.pdf 17.12.15 295
1사분기 통화정책 불확실성 확대 가능성 경계 미래에셋증권 http://imgstock.naver.com/upload/research/debenture/1513231547466.pdf 17.12.14 279
금리를 인상한 연준 2018년은 어떨까? 이베스트증권 http://imgstock.naver.com/upload/research/debenture/1513216121339.pdf 17.12.14 385
이제는 집에 가야 할 시간  하나증권 http:

incorrect startxref pointer(2)


2018년 트렌드 점검 (1) 한-미 금리 역전 본격화 미래에셋증권 http://imgstock.naver.com/upload/research/debenture/1512525889409.pdf 17.12.06 562


FloatObject (b'411.7177.121') invalid; use 0.0 instead
FloatObject (b'102.0821.3559') invalid; use 0.0 instead
FloatObject (b'122.24213.84') invalid; use 0.0 instead
FloatObject (b'0821.35577.121') invalid; use 0.0 instead
FloatObject (b'475.438.6') invalid; use 0.0 instead
FloatObject (b'104.010.84') invalid; use 0.0 instead
FloatObject (b'104.96149.3203') invalid; use 0.0 instead
FloatObject (b'458.8118.84') invalid; use 0.0 instead
FloatObject (b'282.23.48') invalid; use 0.0 instead
FloatObject (b'109.460.68') invalid; use 0.0 instead
FloatObject (b'282.23..121') invalid; use 0.0 instead
FloatObject (b'10011.7177.12') invalid; use 0.0 instead
FloatObject (b'101.5.281') invalid; use 0.0 instead
FloatObject (b'28.2477.238') invalid; use 0.0 instead
FloatObject (b'43..121') invalid; use 0.0 instead
FloatObject (b'477.444.44') invalid; use 0.0 instead
FloatObject (b'28.2477.23892') invalid; use 0.0 instead
FloatObject (b'28.2477.231.5.281') invalid; use 0.0 instead
FloatObject (b'466..2

중단기물 중심의 강세 이어질 전망 현대차증권 http://imgstock.naver.com/upload/research/debenture/1512352538538.pdf 17.12.04 387
변동성에 약한 크레딧시장 현대차증권 http://imgstock.naver.com/upload/research/debenture/1512352341052.pdf 17.12.04 295
11월 금통위 Review 신중한 금리인상  이베스트증권 http://imgstock.naver.com/upload/research/debenture/1512106459130.pdf 17.12.01 318
금리인상의 기다림은 끝나고 투자에 임할 때 미래에셋증권 http://imgstock.naver.com/upload/research/debenture/1512106155114.pdf 17.12.01 436
2018년 완만한 기준금리 인상 속도 예상 미래에셋증권 http://imgstock.naver.com/upload/research/debenture/1512105801382.pdf 17.12.01 347
소신이 살아있다 현대차증권 http://imgstock.naver.com/upload/research/debenture/1512092225571.pdf 17.12.01 293
미국채 금리, 경기회복 기대감 높아지면서 상승 교보증권 http://imgstock.naver.com/upload/research/debenture/1512007294369.pdf 17.11.30 336
11월 금통위 Preview, 박빙이지만 동결을 예상한다 이베스트증권 http://imgstock.naver.com/upload/research/debenture/1512005568380.pdf 17.11.30 288
국내 채권시장, 초장기물 중심으로 강세 교보증권 http://imgstock.naver.com/upload/research/debenture/1511933628397.pdf 17.11.29 329
여전

incorrect startxref pointer(2)
Object 47 0 not defined.
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 47 0 not defined.
Overwriting cache for 0 47
Object 4

RecursionError: maximum recursion depth exceeded while calling a Python object